In [1]:
import numba
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

In [2]:
def dist(a, b):
    return np.sqrt((a - b)**2)

#distancia euclideana entre 2 puntos de A y B: np.linalg.norm(A[i]-B[j])

def max_min_dist(X, Y):
    m,n = X.shape
    maxs = np.empty(m)
    for k in np.arange(m):
        mins = np.empty((m,n))
        for i in np.arange(m):
            mins[i] = np.linalg.norm(X[k]-Y[i])
        maxs[k] = np.amin(mins[k,:])
    return np.amax(maxs)   
    
def hausdorff(X, Y):
    return np.maximum(max_min_dist(X, Y), max_min_dist(Y, X))

In [3]:
@numba.jit('float64 (float64, float64)',nopython=True)
def numba_dist(a, b):
    return np.sqrt((a - b)**2)

@numba.jit('float64 (float64[:,:], float64[:,:])',nopython=True)
def numba_max_min_dist(X, Y):
    m,n = X.shape
    maxs = np.empty(m)
    for k in np.arange(m):
        mins = np.empty((m,n))
        for i in np.arange(m):
            mins[i] = np.linalg.norm(X[k]-Y[i])
        maxs[k] = np.amin(mins[k,:])
    return np.amax(maxs)    

@numba.jit('float64 (float64[:,:], float64[:,:])',nopython=True)
def numba_hausdorff(X, Y):
    return np.maximum(numba_max_min_dist(X, Y), numba_max_min_dist(Y, X))

In [4]:
for i in np.arange(1,11):
    n = i*100
    m = 100
    X = np.random.random((n,m))
    Y = np.random.random((n,m))
    print("Iteración " + str(i) +":")
    %timeit hausdorff(X,Y)

Iteración 1:
10 loops, best of 3: 166 ms per loop
Iteración 2:
1 loop, best of 3: 563 ms per loop
Iteración 3:
1 loop, best of 3: 1.24 s per loop
Iteración 4:
1 loop, best of 3: 2.45 s per loop
Iteración 5:
1 loop, best of 3: 3.15 s per loop
Iteración 6:
1 loop, best of 3: 5.36 s per loop
Iteración 7:
1 loop, best of 3: 7.21 s per loop
Iteración 8:
1 loop, best of 3: 9.05 s per loop
Iteración 9:
1 loop, best of 3: 12.6 s per loop
Iteración 10:
1 loop, best of 3: 17.6 s per loop


In [5]:
for i in np.arange(1,11):
    n = i*100
    m = 100
    X = np.random.random((n,m))
    Y = np.random.random((n,m))
    print("Iteración " + str(i) +" con Numba:")
    %timeit numba_hausdorff(X,Y)

Iteración 1 con Numba:
10 loops, best of 3: 23.7 ms per loop
Iteración 2 con Numba:
10 loops, best of 3: 46.7 ms per loop
Iteración 3 con Numba:
10 loops, best of 3: 103 ms per loop
Iteración 4 con Numba:
10 loops, best of 3: 204 ms per loop
Iteración 5 con Numba:
1 loop, best of 3: 363 ms per loop
Iteración 6 con Numba:
1 loop, best of 3: 410 ms per loop
Iteración 7 con Numba:
1 loop, best of 3: 554 ms per loop
Iteración 8 con Numba:
1 loop, best of 3: 709 ms per loop
Iteración 9 con Numba:
1 loop, best of 3: 925 ms per loop
Iteración 10 con Numba:
1 loop, best of 3: 1.08 s per loop


Luego de comparar los tiempos de ejecución podemos concluir de estos que la ejecución con JIT obviamente es más rápida, pero también podemos ver que la variación respecto al tamaño de la entrada es mejor utilizando JIT, realizando el cuociente entre el último tiempo de ejecución con el primero para python con y sin JIT, tenemos los siguientes valores aprox:

- Python sin JIT: 106.024
- Python con JIT: 45.567